In [1]:
import pandas as pd
import json as js 
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
train_data = js.load(open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/small_mfe_train_data.json'))

In [3]:
train_vid = []
for item in train_data:
    train_vid.append(item['vid'])

In [5]:
js.dump(train_vid,open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/small_mfe_train_data_vid.json','w'))

In [2]:
train_vid = js.load(open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/small_mfe_train_data_vid.json'))

# 载入甘道夫数据

In [ ]:
# 载入一个csv文件
csv_path = '/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/new_ori_data/dataset_20230901.csv'
save_path = '/mnt/bn/luoruipu-disk/meta_data/sft_data/mfe_gandalf/dataset_20230901.json'
drop_feature = js.load(open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/drop_features.json'))
general_feature_list = js.load(open('/mnt/bn/luoruipu-disk/code_base/valley/valley_bussiness/data_process/general_feature.json'))
gandalf_feature_list = js.load(open('/mnt/bn/luoruipu-disk/code_base/valley/valley_bussiness/data_process/gandalf_feature.json'))
df_iterator = pd.read_csv(csv_path, chunksize = 1000,low_memory=False)
pbar = tqdm(total=len(df_iterator))
pbar.set_description('download')
update = lambda *args: pbar.update()
p = Pool(int(16))  # 指定进程池中的进程数
for index,data in enumerate(tqdm(df_iterator)):
     p.apply_async(process_data, args = (data,train_vid, save_path, index), callback=update)
     break
print('Waiting for all subprocesses done...')
p.close()
p.join() 
print('All subprocesses done.')
    

0it [00:00, ?it/s]

0it [00:04, ?it/s]


In [ ]:
def process_data(data, train_vid, save_path, index):
    data = data.replace('\\N',-1.0)
    add_feature = [i for i in data.columns if i not in drop_feature]
    # 填充na
    new_data_orig = pd.DataFrame(data, columns=add_feature)
    values = {key: -1.0 for key in gandalf_feature_list}
    new_data = new_data_orig.iloc[:,9:].fillna(value=values,inplace=False)
    
    feature_matrix = new_data.values.astype(np.float32)
    # courtry data
    country_dummy = pd.get_dummies(data['country']).astype(np.float32)
    country_matrix = country_dummy.values
    # feature concat
    feature_final = np.concatenate((feature_matrix,country_matrix),axis=1)
    general_feature = new_data_orig[['vid','anchor_title','video_desp','audit_result_code','country','asr','merge_ocr']] 
    reject_reason = data[['reject_reason']]
    general_feature_df = pd.concat([general_feature,reject_reason],axis=1)
    result = process_feature(general_feature_df, feature_final, train_vid)
    js.dump(result, open(save_path+f'.part{index}','w'))

In [24]:
def process_feature(general_feature_df, gandalf_feature, train_vid):
    number_row = general_feature_df.shape[0]
    assert number_row == gandalf_feature.shape[0]
    result = []
    for i in range(0, number_row):
        
        general_feature_dict = dict(general_feature_df.iloc[i])
        if general_feature_dict['vid'] in train_vid:
            gandalf_feature_vector = gandalf_feature[i].tolist()
            general_feature_dict['gandalf_vector'] = gandalf_feature_vector
            result.append(general_feature_dict)
    return result

In [25]:
general_feature_df

,vid,anchor_title,video_desp,audit_result_code,country,asr,merge_ocr,reject_reason
0,7273706447471512837,HANDBODY SUPREM SUPER WHITENI,NaN,0,ID,NaN,Bayar di tempat HANDBODY SUPREM SUPER WHITENIN...,Misleading Functionality and Effect
1,7273709588736773381,SUKEE Hair Removal Cream 60g w,NaN,0,PH,hala ka naman,SUKEE SKIN ESSENTIALS SUKEE SKIN ESSENTIALS HI...,Body Impurities
2,7273846802019798277,Shop037 PROMO Original PARAGIS,#foryourpage #fyp #foryoupage #paragis origina...,0,PH,NaN,Cash on Delivery Price Discount Everything mus...,Misleading Functionality and Effect
3,7273864045382356229,Shop037 PROMO Original PARAGIS,#paragis original capsule #foryourpage #fyp #f...,1,PH,NaN,Free Shipping Kidney Stone Cash on Delivery Pr...,NaN
4,7273834176200281349,สั่งซื้อตรงนี้เลยค่า,สเวตเตอร์เกาหลีเกาใจตัวละ 75 บาททั้งร้าน อันยอ...,1,TH,NaN,โปรวันแรกของเดือน 75บาททุกตัวไปเลยค่าา รหัส:BK...,NaN
...,...,...,...,...,...,...,...,...
995,7273948787888934146,buy it here,Have you ever tried this eyelash hack?This hac...,1,MY,NaN,NaN,NaN
996,7273950216091438337,buy it here,Have you ever tried this eyelash hack?This hac...,1,MY,NaN,NaN,NaN
997,7273951668188892418,buy it here,Have you ever tried this eyelash hack?This hac...,1,MY,NaN,NaN,NaN
998,7273838909916810502,ชิโรอิ โปร 1 กระปุก กลูต้า เบอ โลชั่น AHA sol...,null#voiceeffects รีวิวความvาว@บอกต่อของดี @บอ...,0,TH,อันนี้คือของที่ใช้จริงแล้วก็รีวิวจริงจริงเลยนะ...,DEEP CLEANSING AURA WHITS AURA 60pcs 牛油果植萃卸妆湿巾...,Promoting Counterfeit Products


In [15]:
new_data_orig.iloc[:,:9] 

,creator_id,audit_result_code,country,vid,anchor_title,video_desp,video_frame,asr,merge_ocr
0,1690234041545778,0,ID,7273706447471512837,HANDBODY SUPREM SUPER WHITENI,NaN,"[""http://image-va.byted.org/storage/v1/tos-mal...",NaN,Bayar di tempat HANDBODY SUPREM SUPER WHITENIN...
1,180467956630392832,0,PH,7273709588736773381,SUKEE Hair Removal Cream 60g w,NaN,"[""http://image-va.byted.org/storage/v1/tos-mal...",hala ka naman,SUKEE SKIN ESSENTIALS SUKEE SKIN ESSENTIALS HI...
2,180467956630392832,0,PH,7273846802019798277,Shop037 PROMO Original PARAGIS,#foryourpage #fyp #foryoupage #paragis origina...,"[""http://image-va.byted.org/storage/v1/tos-mal...",NaN,Cash on Delivery Price Discount Everything mus...
3,180467956630392832,1,PH,7273864045382356229,Shop037 PROMO Original PARAGIS,#paragis original capsule #foryourpage #fyp #f...,"[""http://image-va.byted.org/storage/v1/tos-mal...",NaN,Free Shipping Kidney Stone Cash on Delivery Pr...
4,6508655644160442369,1,TH,7273834176200281349,สั่งซื้อตรงนี้เลยค่า,สเวตเตอร์เกาหลีเกาใจตัวละ 75 บาททั้งร้าน อันยอ...,"[""http://image-va.byted.org/storage/v1/tos-mal...",NaN,โปรวันแรกของเดือน 75บาททุกตัวไปเลยค่าา รหัส:BK...
...,...,...,...,...,...,...,...,...,...
995,7264515457833632774,1,MY,7273948787888934146,buy it here,Have you ever tried this eyelash hack?This hac...,"[""http://image-sg.byted.org/storage/v1/tos-ali...",NaN,NaN
996,7264515457833632774,1,MY,7273950216091438337,buy it here,Have you ever tried this eyelash hack?This hac...,"[""http://image-sg.byted.org/storage/v1/tos-ali...",NaN,NaN
997,7264515457833632774,1,MY,7273951668188892418,buy it here,Have you ever tried this eyelash hack?This hac...,"[""http://image-sg.byted.org/storage/v1/tos-ali...",NaN,NaN
998,7264793766971294725,0,TH,7273838909916810502,ชิโรอิ โปร 1 กระปุก กลูต้า เบอ โลชั่น AHA sol...,null#voiceeffects รีวิวความvาว@บอกต่อของดี @บอ...,"[""http://image-va.byted.org/storage/v1/tos-mal...",อันนี้คือของที่ใช้จริงแล้วก็รีวิวจริงจริงเลยนะ...,DEEP CLEANSING AURA WHITS AURA 60pcs 牛油果植萃卸妆湿巾...


# 找到gandalf 测试集的 vid

In [1]:
import json as js
gandalf_test_data = js.load(open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/small_mfe_test_data.json'))
test_vid = [data['vid'] for data in gandalf_test_data]
js.dump(test_vid, open('/mnt/bn/ecom-lxy/zhuhe/valley/gpt_data/small_mfe_test_data_vid.json','w'))

'7275174989966494981'